# Solución tarea semana 4

## 1. Introducción	
    En esta tarea se dará solución para llevar a cabo los diferentes procesos de ETL para la dimensión GeografiaConDemografia, se partirá de un diseño que permite orientar la implementación y tener en cuenta las consideraciones del proceso ETL como los campos a tener en cuenta y las transformaciones sobre las reglas o necesidades de negocio que permitan obtener análisis de una forma más fiable y entendible.
	

## 2. Proceso de ETL.

En este proceso de ETL, se extraen los datos de los **pib, divipola y proyecciones** de una base de datos transaccional y se almacenan en otra base de datos que corresponde a la bodega de datos, siguiendo una aproximación ROLAP. A continuación, se presenta el modelo multidimensional que es el modelo conceptual que representa el proceso de registro de vuelos en un mes para la demografía y la geografía. Este modelo se utilizará para crear la dimensión GeografiaConDemografia en la bodega de datos que representan el proceso de negocio y que será cargada como resultado del proceso ETL. 

Se tendrán dos llaves las cuales están relacionadas con la bodega de datos (terminadas en DWH) y con las llaves de la base de datos transaccional (terminadas en T).


![Modelo](./modelo.png)

Para el proceso de ETL se realizó el siguiente diseño general. A partir de la información obtenida del diseño ETL, se pudieron identificar las fuentes a utilizar y las relaciones entre estas, también la organización ha dado respuesta a algunas inquietudes, lo que permite conocer la mejor forma de proceder para manipular los datos. En el proceso de ETL se plantea un bloque para la dimensión nombrada anteriormente.
![ETL](./Diseno-ETL.png)

### Configuración inicial para la sesión y base de datos

In [1]:
# Configuración servidor base de datos transaccional
db_user = ''
db_psswd = ''
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'

dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Estudiante_81_202413'

# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [2]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace, to_timestamp
from datetime import datetime

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)
spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### Conexión y carga de datos

Se define la función para conexión y cargue de dataframes desde la base de datos origen y luego la función para guardar un dataframe en una tabla de la base de datos destino.

In [4]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

## BLOQUE 1
Empezamos con el bloque 1: primero se procederá a obtener la información de las fuentes de datos pib, divipola y proyecciones, luego se realizarán las transformaciones para normalizar los nombres de los departamentos y eliminar duplicados de la tabla pib, pra realizar imputaciones en la tabla de proyecciones y por último hacer la intregración de las tres fuentes de datos para cargar la información en la dimensión **GeografiaConDemografia**

### Extracción
Se toman la información de las 3 tablas mencionadas anteriormente del modelo transaccional y se almacenan como dataframes.

In [130]:
sql_pib = '''(SELECT DISTINCT `CodigoDepartamento(DIVIPOLA)` as codDepartamentoPib, DEPARTAMENTOS ,`2006` ,`2007`, `2008`, `2010`, `2011`, `2012`, `2013`, `2014`, `2015`, `2016`, `2017`, `2009`, `2005` FROM ProyectoTransaccional.pib) AS Temp_pib'''

pib = obtener_dataframe_de_bd(source_db_connection_string, sql_pib, db_user, db_psswd)

In [160]:
sql_divipola = '''(SELECT DISTINCT CodigoDepartamento, CodigoMunicipio, NombreDepartamento, NombreMunicipio, NombreAreaMetropolitana, Longitud, Latitud FROM ProyectoTransaccional.divipola) AS Temp_divipola'''
divipola = obtener_dataframe_de_bd(source_db_connection_string, sql_divipola, db_user, db_psswd)

In [230]:
sql_proyecciones = '''(SELECT DISTINCT DP, AÑO, `Total Hombres`, `Total Mujeres` FROM ProyectoTransaccional.proyecciones) AS Temp_proyecciones'''
proyecciones = obtener_dataframe_de_bd(source_db_connection_string, sql_proyecciones, db_user, db_psswd)

In [201]:
sql_aeropuertos = '''(SELECT DISTINCT gcd_departamento, gcd_municipio CodigoMunicipio, latitud, longitud FROM ProyectoTransaccional.aeropuertos) AS Temp_aeropuertos'''
aeropuertos = obtener_dataframe_de_bd(source_db_connection_string, sql_aeropuertos, db_user, db_psswd)

In [133]:
print(pib.columns, divipola.columns, proyecciones.columns)

['codDepartamentoPib', 'DEPARTAMENTOS', '2006', '2007', '2008', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2009', '2005'] ['CodigoDepartamento', 'CodigoMunicipio', 'NombreDepartamento', 'NombreMunicipio', 'NombreAreaMetropolitana', 'Longitud', 'Latitud'] ['DP', 'AÑO', 'Total Hombres', 'Total Mujeres']


### Transformación
Como parte del proceso de transformación, se procede a realizar lo siguiente:
1. Realizar las transformaciones pertinentes en cada fuente de datos, se establecerá una sección por cada fuente para que quede más ordenado el proceso.  
2. Se realizará la ingración entre las diferentes tabla para que todo quede listo para el cargue.


#### pib
A continuación se muestra el proceso para las transaformaciones de la información en pib, primero se procede a validar la cantidad de registros total y la cantidad de duplicados

In [134]:
print("Cantidad de registros distintos : ",pib.distinct().count(), " Cantidad de registros: ",
      pib.count(), "Cantidad de departamentos distintos: ", pib.select("DEPARTAMENTOS").distinct().count(), 
     "Cantidad de código de departamentos distintos", pib.select("codDepartamentoPib").distinct().count())

Cantidad de registros distintos :  34  Cantidad de registros:  34 Cantidad de departamentos distintos:  34 Cantidad de código de departamentos distintos 28


In [135]:
pib.show()

+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|codDepartamentoPib|       DEPARTAMENTOS|    2006|    2007|                2008|                2010|    2011|    2012|                2013|    2014|    2015|    2016|    2017|    2009|    2005|
+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|                47|        El Magdalena| 4202994| 4618524|   5293554.101685749|   6119382.927756879| 6441145| 6942728|   7530341.883201833| 7620221| 8285062| 9081224| 9488025| 5858657| 3938108|
|                54|            norte de| 4562230| 5093601|   5756904.272168976|   6377160.081745004| 6845992| 7152319|   7716659.361075875| 8310220| 8890010| 9693223| 9984804| 6139384| 3928796|
|                18|     

Al contar la cantidad de registros distintos y el total de registros en la tabla pib, el resultado es el mismo, sin embargo, se puede apreciar que en realidad si existen duplicados, ya que hay códigos de departamentos iguales, pero el nombre del departamente tiene variaciones, como por ejemplo <i>El amazonas</i> y <i>amazX</i>. <br>
Por lo anterior, se procede a estandarizar los nombres de los departamentos, en donde primero se extraerán todos los nombres de departamentos que están mal escritos y se pondrán correctamente y en mayúscula

In [137]:
departamentosDic={'norte de':'EL NORTE DE SANTANDER',
          'CaquetÃ¡':'CAQUETÁ',
          'El ChocÃ³':'EL CHOCÓ', 
          'amazX': 'EL AMAZONAS', 
          'uila': 'HUILA',
          'AtlÃ¡ntico': 'ATLÁNTICO',
          'CÃ³rdoba': 'CÓRDOBA',
          'CAQUETA': 'CAQUETÁ',
          'ATLANTICO': 'ATLÁNTICO',
          'VASANARE': 'CASANARE',
          'QUINDIO': 'QUINDÍO',
          'GUANIA': 'GUAINÍA',
          'boyaca': 'BOYACÁ'
         }
#pib=pib.rdd.map(lambda x: 
#    (x.codDepartamentoPib, departamentosDic.get(x.DEPARTAMENTOS, x.DEPARTAMENTOS),x.2006,x.2007,x.2008,x.2010,x.2011,x.2012,x.2013,x.2014,x.2015,x.2016,x.2017,x.2009,x.2005) 
#    ).toDF(["codDepartamentoPib", "DEPARTAMENTOS" ,"2006" ,"2007" ,"2008" ,"2010" ,"2011" ,"2012" ,"2013" ,"2014" ,"2015" ,"2016" ,"2017" ,"2009" ,"2005"])
pib = pib.withColumn('DEPARTAMENTOS', f.udf(lambda dep: departamentosDic.get(dep, dep))(f.col('DEPARTAMENTOS')))
pib.show()

+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|codDepartamentoPib|       DEPARTAMENTOS|    2006|    2007|                2008|                2010|    2011|    2012|                2013|    2014|    2015|    2016|    2017|    2009|    2005|
+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|                47|        El Magdalena| 4202994| 4618524|   5293554.101685749|   6119382.927756879| 6441145| 6942728|   7530341.883201833| 7620221| 8285062| 9081224| 9488025| 5858657| 3938108|
|                54|EL NORTE DE SANTA...| 4562230| 5093601|   5756904.272168976|   6377160.081745004| 6845992| 7152319|   7716659.361075875| 8310220| 8890010| 9693223| 9984804| 6139384| 3928796|
|                18|     

Ahora que los nombres de los departamentos ya no tienen errores ortográficos, se procederá a poner todos los nombres en mayúsculas

In [139]:
pib.select("DEPARTAMENTOS").distinct().count()

31

Existen menos departamentos distintos, por lo que ya se van haciendo evidentes los duplicados.

In [140]:
pib = pib.withColumn("DEPARTAMENTOS", f.upper("DEPARTAMENTOS"))
pib.show(10)

+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|codDepartamentoPib|       DEPARTAMENTOS|    2006|    2007|                2008|                2010|    2011|    2012|                2013|    2014|    2015|    2016|    2017|    2009|    2005|
+------------------+--------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+
|                47|        EL MAGDALENA| 4202994| 4618524|   5293554.101685749|   6119382.927756879| 6441145| 6942728|   7530341.883201833| 7620221| 8285062| 9081224| 9488025| 5858657| 3938108|
|                54|EL NORTE DE SANTA...| 4562230| 5093601|   5756904.272168976|   6377160.081745004| 6845992| 7152319|   7716659.361075875| 8310220| 8890010| 9693223| 9984804| 6139384| 3928796|
|                18|     

In [142]:
print("Código de departamentos distintos: ", pib.select("codDepartamentoPib").distinct().count(), "Cantidad de departamentos diferentes: ", pib.select("DEPARTAMENTOS").distinct().count())

Código de departamentos distintos:  28 Cantidad de departamentos diferentes:  29


Como se puede evidenciar previamente, la cantidad de códigos distintos y de nombre de departamentos difiere en 1, la razón de esto es porque hay 2 códigos de departaments que están asignados a 2 nombres de departamentos, pero estos en realidad no son nombres de departamentos, el código de departamento 0 tiene asociado a colombia tierra querida y a Republica de Colombia como se puede apreciar a continuación

In [143]:
pib.select("codDepartamentoPib", "DEPARTAMENTOS").where(col("codDepartamentoPib") == 0).show()

+------------------+--------------------+
|codDepartamentoPib|       DEPARTAMENTOS|
+------------------+--------------------+
|                 0|COLOMBIA TIERRA Q...|
|                 0|REPUBLICA DE COLO...|
+------------------+--------------------+



Se procede a no tener en cuenta los código de departamento con valor cero

In [144]:
pib = pib.where(col("codDepartamentoPib") != 0)

In [145]:
print("Código de departamentos distintos: ", pib.select("codDepartamentoPib").distinct().count(), "Cantidad de departamentos diferentes: ", pib.select("DEPARTAMENTOS").distinct().count())

Código de departamentos distintos:  27 Cantidad de departamentos diferentes:  27


In [146]:
pib.select("codDepartamentoPib").distinct().count(), pib.select("DEPARTAMENTOS").distinct().count()

(27, 27)

Ahora sí se puede apreciar que se tienen la misma cantidad de códigos de departamentos y de códigos, pero esto no significa que no existan duplicados, hasta el momento ya se ha estandarizado la información, por lo que es viable eliminar los duplicados, si hacemos un conteo de la cantidad de registros, se observa que existen 32, en Colombia hay 32 departamentos, pero en este caso, hay algunos duplicados

In [147]:
pib.count()

32

In [148]:
pib = pib.drop_duplicates()
pib.count()

27

Como se puede ver, ya no existen registros duplicados en pib

#### divipola
El campo NombreAreaMetropolitana contiene bastantes valores desconocidos o nulos, por este motivo se le asignarán a los valores null la cadena 'desconocido'

In [161]:
cantidadNulos = divipola.filter(divipola["NombreAreaMetropolitana"].isNull()).count()
cantidadDistintos = divipola.select("NombreAreaMetropolitana").distinct().count()
print("Registros nulos: ", cantidadNulos, " Nobre de áreas metropolitana distintos: ", cantidadDistintos)

Registros nulos:  7394  Nobre de áreas metropolitana distintos:  7


In [163]:
divipola.groupBy("NombreAreaMetropolitana").count().show()

+-----------------------+-----+
|NombreAreaMetropolitana|count|
+-----------------------+-----+
|   AREA METROPOLITAN...|   64|
|   AREA METROPOLITAN...|   60|
|   AREA METROPOLITAN...|  128|
|                   null| 7394|
|   AREA METROPOLITAN...|   44|
|   AREA METROPOLITAN...|   11|
|   AREA METROPOLITAN...|   93|
+-----------------------+-----+



Se puede observar la cantidad de registros que existen con cada nombre de área metropolitana y la cantidad de nulos que conincide con el distinct ejecutado en el paso previo, esto es importante para poder ver que al convertir los nulos en el valor <i>desconocido</i> no se vayan a perder datos

Nos podemos dar cuenta que existen 7394 registros vacíos y solo 7 nombre de área metropolitana diferentes.

In [168]:
divipola = divipola.withColumn('NombreAreaMetropolitana', f.when(col('NombreAreaMetropolitana') != "null",col('NombreAreaMetropolitana')).otherwise('desconocido')) 
divipola.show(10)

+------------------+---------------+------------------+---------------+-----------------------+--------------+--------------+
|CodigoDepartamento|CodigoMunicipio|NombreDepartamento|NombreMunicipio|NombreAreaMetropolitana|      Longitud|       Latitud|
+------------------+---------------+------------------+---------------+-----------------------+--------------+--------------+
|                91|          91001|          amazonas|        LETICIA|            desconocido|-69.9414267076|-4.19983693453|
|                91|          91001|          amazonas|        LETICIA|            desconocido|-69.9588125704|-4.19469878932|
|                91|          91001|          amazonas|        LETICIA|            desconocido|-69.9753934295|-4.17750342085|
|                91|          91001|          amazonas|        LETICIA|            desconocido|-69.9288659026|-4.17628953753|
|                91|          91001|          amazonas|        LETICIA|            desconocido|-69.9582013552|-4.15376

In [169]:
cantidadNulos = divipola.filter(divipola["NombreAreaMetropolitana"].isNull()).count()
cantidadDistintos = divipola.select("NombreAreaMetropolitana").distinct().count()
print("Registros nulos: ", cantidadNulos, " Nobre de áreas metropolitana distintos: ", cantidadDistintos)

Registros nulos:  0  Nobre de áreas metropolitana distintos:  7


In [170]:
divipola.groupBy("NombreAreaMetropolitana").count().show()

+-----------------------+-----+
|NombreAreaMetropolitana|count|
+-----------------------+-----+
|   AREA METROPOLITAN...|   64|
|   AREA METROPOLITAN...|   60|
|            desconocido| 7394|
|   AREA METROPOLITAN...|  128|
|   AREA METROPOLITAN...|   44|
|   AREA METROPOLITAN...|   11|
|   AREA METROPOLITAN...|   93|
+-----------------------+-----+



Nos podemos dar cuenta que solo los valores nulos se han cambiado por la palabra **desconodido**, los nombres de áreas metropolitanas continúan intactos

**Obtener información de aeropuerto para la ubicación**

Para tener certeza de la ubicación exacta del municipio del aeropuerto, se procede a comparar la longitud y latitud de los aeropuertos con los centros poblados

In [214]:
totalAeropuertos = aeropuertos.select("*").count() 
totalMunicipiosDiferentes = aeropuertos.select("CodigoMunicipio").distinct().count()
print("Total de aeropuertos distintos registrados: ", totalAeropuertos, " total de municipios diferentes: ", totalMunicipiosDiferentes)

Total de aeropuertos distintos registrados:  595  total de municipios diferentes:  241


In [203]:
cantidadNulosLa = aeropuertos.filter(aeropuertos["latitud"].isNull()).count()
cantidadNulosLon = aeropuertos.filter(aeropuertos["longitud"].isNull()).count()
print("Cantidad de registros nulos para la longitud: ", cantidadNulosLa, "Cantidad de registros nulos para la longitud: ", cantidadNulosLon)

Cantidad de registros nulos para la longitud:  0 Cantidad de registros nulos para la longitud:  0


Los campos de longitud y latitud no poseen registros nulos

In [262]:
joinAerDivi = aeropuertos.alias('a').join(divipola.alias('div'), how = 'left', on = 'CodigoMunicipio')
geografia = joinAerDivi.select("gcd_departamento", "CodigoMunicipio", "NombreMunicipio", "NombreAreaMetropolitana", "a.longitud", "a.latitud").groupBy("gcd_departamento", "CodigoMunicipio", "NombreMunicipio", "NombreAreaMetropolitana", "a.longitud", "a.latitud").count()
geografia.show(10)

+----------------+---------------+--------------------+-----------------------+-----------+---------+-----+
|gcd_departamento|CodigoMunicipio|     NombreMunicipio|NombreAreaMetropolitana|   longitud|  latitud|count|
+----------------+---------------+--------------------+-----------------------+-----------+---------+-----+
|              68|          68575|      PUERTO WILCHES|            desconocido|   -73.7965|   7.3283|   22|
|              68|          68575|      PUERTO WILCHES|            desconocido|   -73.8151|   7.2417|   22|
|              68|          68575|      PUERTO WILCHES|            desconocido|   -73.7464|    7.462|   22|
|              68|          68575|      PUERTO WILCHES|            desconocido|   -73.7723|   7.2466|   22|
|              68|          68575|      PUERTO WILCHES|            desconocido|   -73.7464|   7.4614|   22|
|              68|          68575|      PUERTO WILCHES|            desconocido|-73.8928278|7.3479944|   22|
|              13|          

Como se puede apreciar, se realiza un join entre la tabla de aeropuertos y divipola con el objetivo de obtener la información relacionada al código del departamento, código del municipio, el nombre del área metropolitana, la longitud y latitud del aeropuerto, esto se logra por medio del agrupamieto de estos campos, gracias a esto se puede obtener la longitud y latitud exactas de cada aeropuerto sin importar que en un municipio existan varios aeropuertos

In [263]:
geografia.count(), geografia.select("CodigoMunicipio").distinct().count()

(595, 241)

La cantidad de registros es de 595 para geografía y de municipios distintos sigue siendo 241, esto es congruente con la cantidad de aeropuertos y municipios distintos que se recuperó de base de datos, como se mostró en un paso previo.

#### proyecciones

Se procederá a realizar las imputaciones sobre proyecciones las cuales corresponden a imputar el total de mujeres para el valor máximo y a eliminar los duplicados de la tabla

In [231]:
proyecciones.count()

1287

En la base de datos proyectoTransaccional existen 2574 registros, con el resultado anterior, se puede ver que al traer la información de la tabla proyecciones haciendo uso de distinct, los registros ya no vienen duplicados

In [249]:
proyecciones = proyecciones.orderBy(col("Total Mujeres").desc())
proyecciones.show(10)

+---+----+-------------+-------------+
| DP| AÑO|Total Hombres|Total Mujeres|
+---+----+-------------+-------------+
| 99|2009|        47300|     55000000|
| 11|2017|      3511579|      3825870|
| 11|2017|      3499357|      3813887|
| 11|2016|      3495609|      3805309|
| 11|2016|      3483805|      3793738|
| 11|2015|      3484883|      3788382|
| 11|2015|      3473470|      3777205|
| 11|2014|      3476538|      3776411|
| 11|2014|      3465493|      3765604|
| 11|2013|      3466060|      3762367|
+---+----+-------------+-------------+
only showing top 10 rows



Imputar el valor máximo de total mujeres

In [250]:
proyecciones.select('Total Mujeres').sort(col("Total Mujeres").desc()).collect()[0]

Row(Total Mujeres=55000000)

In [252]:
proyeccionesConImputacion = proyecciones.replace( 55000000, 39691, 'Total Mujeres')
proyeccionesConImputacion.show(10)

+---+----+-------------+-------------+
| DP| AÑO|Total Hombres|Total Mujeres|
+---+----+-------------+-------------+
| 99|2009|        47300|        39691|
| 11|2017|      3511579|      3825870|
| 11|2017|      3499357|      3813887|
| 11|2016|      3495609|      3805309|
| 11|2016|      3483805|      3793738|
| 11|2015|      3484883|      3788382|
| 11|2015|      3473470|      3777205|
| 11|2014|      3476538|      3776411|
| 11|2014|      3465493|      3765604|
| 11|2013|      3466060|      3762367|
+---+----+-------------+-------------+
only showing top 10 rows



In [253]:
proyeccionesConImputacion.count()

1287

El valor 39691 se obtiene de la resta entre 86991 y 47300, el primero es del campo total que dio en el registro con el total de mujeres más alto, y el segundo es el total de hombres, también se puede observar que la cantidad de registros para proyecciones sigue siendo 1287

#### Integración
Se procederá a realizar la integración entre los datos obtenidos de proyecciones, pib y la información de geografía obtenida de la tabla divipola y aeropuertos, esta última principalmente para la longitud y latitud

In [276]:
proyeccionesModificadoCampo = proyeccionesConImputacion.withColumnRenamed('AÑO','anio') \
                    .withColumnRenamed('Total Hombres','totalHombres') \
                    .withColumnRenamed('Total Mujeres','totalMujeres')
proyeccionesModificadoCampo.show(10)

+---+----+------------+------------+
| DP|anio|totalHombres|totalMujeres|
+---+----+------------+------------+
| 99|2009|       47300|       39691|
| 11|2017|     3511579|     3825870|
| 11|2017|     3499357|     3813887|
| 11|2016|     3495609|     3805309|
| 11|2016|     3483805|     3793738|
| 11|2015|     3484883|     3788382|
| 11|2015|     3473470|     3777205|
| 11|2014|     3476538|     3776411|
| 11|2014|     3465493|     3765604|
| 11|2013|     3466060|     3762367|
+---+----+------------+------------+
only showing top 10 rows



In [293]:
#proyeccionesConImputacion = proyeccionesConImputacion.withColumnRenamed('DP','CodigoDepartamento')
#geografia = geografia.withColumnRenamed('gcd_departamento','CodigoDepartamento')
#pib
GeografiaConDemografia = geografia.alias('g').join(pib.alias('pib'), geografia.gcd_departamento == pib.codDepartamentoPib,'left')\
                    .join(proyeccionesModificadoCampo.alias('pro'), pib.codDepartamentoPib == proyeccionesModificadoCampo.DP,'left') \
                    .select([col('g.CodigoMunicipio'),col('g.NombreMunicipio'),col('pib.DEPARTAMENTOS'),
                             col('g.NombreAreaMetropolitana'),col('g.longitud'),col('g.latitud'),
                             col('pro.totalHombres'),col('pro.totalMujeres'), col('pro.anio')]) \
                    .fillna({'DEPARTAMENTOS': 'desconocido'})
GeografiaConDemografia.show()

+---------------+---------------+-------------+-----------------------+--------+-------+------------+------------+----+
|CodigoMunicipio|NombreMunicipio|DEPARTAMENTOS|NombreAreaMetropolitana|longitud|latitud|totalHombres|totalMujeres|anio|
+---------------+---------------+-------------+-----------------------+--------+-------+------------+------------+----+
|           5847|          URRAO|    ANTIOQUIA|            desconocido|-76.1409| 6.3273|     1893416|     2143679|2005|
|           5847|          URRAO|    ANTIOQUIA|            desconocido|-76.1409| 6.3273|      737371|      686377|2005|
|           5847|          URRAO|    ANTIOQUIA|            desconocido|-76.1409| 6.3273|     2630787|     2830056|2005|
|           5847|          URRAO|    ANTIOQUIA|            desconocido|-76.1409| 6.3273|     1927511|     2178637|2006|
|           5847|          URRAO|    ANTIOQUIA|            desconocido|-76.1409| 6.3273|      736446|      685527|2006|
|           5847|          URRAO|    ANT

In [314]:
anios = GeografiaConDemografia.select("anio").rdd.flatMap(lambda x: x).collect()
departamentos = GeografiaConDemografia.select("DEPARTAMENTOS").rdd.flatMap(lambda x: x).collect()
depAnios = {
    "2006": 0,
    "2007": 1,
    "2008": 2,
    "2010": 3,
    "2011": 4,
    "2012": 5,
    "2013": 6,
    "2014": 7,
    "2015": 8,
    "2016": 9,
    "2017": 10,
    "2009": 11,
    "2005": 12
}
departamentosDistintos = pib.select("DEPARTAMENTOS").distinct().rdd.flatMap(lambda x: x).collect() 
departamentosYAniosGG = {}
pibList = []
#for idx, anio in enumerate(anios):
#    dep = departamentos[idx]
#    pibNumber = pib.select(f.lit(anio)).where(col("DEPARTAMENTOS") == f.lit(dep)).rdd.flatMap(lambda x: x).collect() 
#    pibList.append(pibNumber)
#pibNumber = pib.select("2006").where(col("DEPARTAMENTOS") == "ANTIOQUIA").rdd.flatMap(lambda x: x).collect() 
for dep in departamentosDistintos:
    pibNumber = pib.select("*").where(col("DEPARTAMENTOS") == f.lit(dep)).rdd.flatMap(lambda x: x).collect()
    departamentosYAniosGG[pibNumber[1]] = pibNumber[2:]
    
#pibNumber = pib.select("2006").where(col("DEPARTAMENTOS") == "ANTIOQUIA").rdd.flatMap(lambda x: x).collect() 
print(departamentosYAniosGG)
    

{'SANTANDER': [11318386, 12762161, 14545114.37228981, 16392991.177437823, 18726378, 19764532, 21093453.800777685, 24482373, 24796146, 26029905, 27702140, 14433162, 9951651], 'CÓRDOBA': [4362011, 5186748, 5129321.435273318, 5503257.383173957, 5893042, 6439647, 6914460.222596752, 7555705, 7910948, 8405654, 8957509, 5235480, 3912903], 'EL MAGDALENA': [4202994, 4618524, 5293554.101685749, 6119382.927756879, 6441145, 6942728, 7530341.883201833, 7620221, 8285062, 9081224, 9488025, 5858657, 3938108], 'BOYACÁ': [8046451, 9523137, 11059677.686639624, 12422311.92528656, 14744101, 15572164, 16401817.15011036, 17552741, 18575739, 19726304, 20520809, 11433547, 7211579], 'EL AMAZONAS': [4169814, 4486342, 4807017.609841224, 5591516.361706729, 5969210, 6370295, 6803987.471579608, 7465703, 8186030, 8952717, 9455515, 5202908, 3830908], 'CASANARE': [24864712, 23554324, 28528736.36597076, 28892100.448943723, 39124122, 41847958, 43845228.45779656, 42843277, 33571338, 28911791, 31964165, 25845154, 24123828]

In [320]:
pibList = []
for idx, dep in enumerate(departamentos):
    pibs = departamentosYAniosGG.get(dep, [])
    if pibs == []:
        continue
    pibList.append(pibs[depAnios[str(anios[idx])]])
print(pibList[:10], len(pibList))

[9340469, 9340469, 9340469, 10362481, 10362481, 10362481, 11530203, 11530203, 11530203, 12084731.97289277] 18018


In [328]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window, SQLContext 

a = GeografiaConDemografia
pibListDouble = map(double, pibList)
b = sql_context.createDataFrame([(l,) for l in pibListDouble], ['PIB'])

#add 'sequential' index and join both dataframe to get the final result
a = a.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

final_df = a.join(b, a.row_idx == b.row_idx).\
             drop("row_idx")

In [339]:
GeografiaConDemografiaFinal = final_df.withColumnRenamed('CodigoMunicipio','idMunicipio_T') \
                                    .withColumnRenamed('DEPARTAMENTOS','nombreDepartamento')
GeografiaConDemografiaFinal = GeografiaConDemografiaFinal.drop('anio')
GeografiaConDemografiaFinal.orderBy(col("totalMujeres").desc()).show()

+-------------+--------------------+------------------+-----------------------+--------+---------+------------+------------+--------------------+
|idMunicipio_T|     NombreMunicipio|nombreDepartamento|NombreAreaMetropolitana|longitud|  latitud|totalHombres|totalMujeres|                 PIB|
+-------------+--------------------+------------------+-----------------------+--------+---------+------------+------------+--------------------+
|         5001|            MEDELLÍN|         ANTIOQUIA|   AREA METROPOLITAN...|-75.5904|   6.2204|     3041835|     3255008|1.3099961280563224E7|
|         5847|               URRAO|         ANTIOQUIA|            desconocido|-76.1409|   6.3273|     3041835|     3255008|1.3099961280563224E7|
|         5585|         PUERTO NARE|         ANTIOQUIA|            desconocido|-74.5902|   6.2102|     3041835|     3255008|1.3099961280563224E7|
|         5480|              MUTATÁ|         ANTIOQUIA|            desconocido|-76.4442|   7.2505|     3041835|     3255008|

Todas las transformaciones hechas anteriormente son con el fin de obtener la información de pib para todos los registros generados, primero se obtienen los pib, luego los departamentos y en base a esto se van realizando los reemplazos hasta completar la tabla

In [352]:
GeografiaConDemografiaFinal = GeografiaConDemografiaFinal.coalesce(1).withColumn('idMunicipio_DWH', f.monotonically_increasing_id() + 1)

In [345]:
GeografiaConDemografiaFinal_0 = [(0,0,'Missing','Missing','Missing',0,0,0,0,0)]
columns = ['idMunicipio_DWH', 'idMunicipio_T','NombreMunicipio','nombreDepartamento','NombreAreaMetropolitana','longitud','latitud','totalHombres','totalMujeres', 'PIB']
GeografiaConDemografiaFinal_0 = spark.createDataFrame(data=GeografiaConDemografiaFinal_0,schema=columns)
GeografiaConDemografiaFinal_0.show()

+---------------+-------------+---------------+------------------+-----------------------+--------+-------+------------+------------+---+
|idMunicipio_DWH|idMunicipio_T|NombreMunicipio|nombreDepartamento|NombreAreaMetropolitana|longitud|latitud|totalHombres|totalMujeres|PIB|
+---------------+-------------+---------------+------------------+-----------------------+--------+-------+------------+------------+---+
|              0|            0|        Missing|           Missing|                Missing|       0|      0|           0|           0|  0|
+---------------+-------------+---------------+------------------+-----------------------+--------+-------+------------+------------+---+



In [354]:
GeografiaConDemografiaFinalF = GeografiaConDemografiaFinal.union(GeografiaConDemografiaFinal_0)
GeografiaConDemografiaFinalF.orderBy(col("totalMujeres").desc()).show()

+-------------+--------------------+------------------+-----------------------+--------+---------+------------+------------+--------------------+---------------+
|idMunicipio_T|     NombreMunicipio|nombreDepartamento|NombreAreaMetropolitana|longitud|  latitud|totalHombres|totalMujeres|                 PIB|idMunicipio_DWH|
+-------------+--------------------+------------------+-----------------------+--------+---------+------------+------------+--------------------+---------------+
|         5001|            MEDELLÍN|         ANTIOQUIA|   AREA METROPOLITAN...|-75.5904|   6.2204|     3041835|     3255008|1.3099961280563224E7|            445|
|         5847|               URRAO|         ANTIOQUIA|            desconocido|-76.1409|   6.3273|     3041835|     3255008|1.3099961280563224E7|            172|
|         5585|         PUERTO NARE|         ANTIOQUIA|            desconocido|-74.5902|   6.2102|     3041835|     3255008|1.3099961280563224E7|            211|
|         5480|             

Se puede observar que se ha creado el registro con valor 0 para asociar con registros faltantes.

### Carga
Una vez realizado el proceso de carga y transformación, se guardan los resultados en la base de datos destino


In [355]:
# CARGUE
guardar_db(dest_db_connection_string, GeografiaConDemografiaFinalF,'Proyecto_G26_202413.GeografiaConDemografia', db_user, db_psswd)

# Resultado de consultas
Corresponde a la consulta realizada sobre la tabla GeografiaConDemografia, para mostrar el estado final de las tablas pobladas como resultado del proceso de ETL.

## Consulta para ver algunos registros de la dimensión Fecha

In [358]:
sql_geodemo = '''(SELECT * FROM Proyecto_G26_202413.GeografiaConDemografia) AS Temp_GeografiaConDemografia'''
sql_geodemoS6 = obtener_dataframe_de_bd(source_db_connection_string, sql_geodemo, db_user, db_psswd)
sql_geodemoS6.orderBy(col("totalMujeres").desc()).show(10)

+-------------+---------------+------------------+-----------------------+--------+-------+------------+------------+--------------------+---------------+
|idMunicipio_T|NombreMunicipio|nombreDepartamento|NombreAreaMetropolitana|longitud|latitud|totalHombres|totalMujeres|                 PIB|idMunicipio_DWH|
+-------------+---------------+------------------+-----------------------+--------+-------+------------+------------+--------------------+---------------+
|         5591| PUERTO TRIUNFO|         ANTIOQUIA|            desconocido|-74.7258| 5.9222|     3041835|     3255008|1.3099961280563224E7|            523|
|         5154|       CAUCASIA|         ANTIOQUIA|            desconocido|-75.1982|  7.968|     3041835|     3255008|1.3099961280563224E7|            913|
|         5591| PUERTO TRIUNFO|         ANTIOQUIA|            desconocido|-74.5742| 5.9936|     3041835|     3255008|1.3099961280563224E7|            562|
|         5847|          URRAO|         ANTIOQUIA|            desconoc